In [1]:
# This is part-2 of my back-testing code to test the momentum stock picking process
# The update will add 2024 returns to the stocks picked in 2023
# The code will pick top-10 stocks from last 4 years based on momentum strength and momentum quality


import numpy as np
import yfinance as yf
import pandas as pd

def get_sp500_tickers():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    tickers = table[0]['Symbol'].tolist()
    return tickers

def fetch_and_analyze_data(ticker, start_date, end_date):
    try:
        adjusted_ticker = ticker.replace('.', '-')
        stock_data = yf.Ticker(adjusted_ticker).history(start=start_date, end=end_date)

        if not stock_data.empty:
            stock_data.index = stock_data.index.tz_localize(None)
            
            start_price = stock_data.iloc[0]['Close']
            end_price = stock_data.iloc[-1]['Close']
            annual_return = (end_price - start_price) / start_price * 100
            
            stock_data['Daily Return'] = stock_data['Close'].pct_change()
            std_dev = stock_data['Daily Return'].std() * (252**0.5)
            sharpe_ratio = (annual_return / 100 - risk_free_rate) / std_dev
            
            pe_ratio = yf.Ticker(adjusted_ticker).info.get('trailingPE', None)

            stock_data['up_days'] = stock_data['Daily Return'] > 0
            stock_data['down_days'] = stock_data['Daily Return'] < 0

            up_days_percentage = stock_data['up_days'].sum() / len(stock_data) * 100
            down_days_percentage = stock_data['down_days'].sum() / len(stock_data) * 100

            stock_data['Momentum Health Indicator'] = up_days_percentage - down_days_percentage
            stock_data.drop(columns=['up_days', 'down_days'], inplace=True)
            
            expected_return = stock_data['Daily Return'].mean()
            var_95 = expected_return - 1.96 * std_dev
            stock_data['VAR'] = var_95

            return {'Ticker': ticker, 'Return': annual_return, 'Std Dev': std_dev, 'Sharpe Ratio': sharpe_ratio, 'P/E Ratio': pe_ratio, 'Momentum Health Indicator': stock_data['Momentum Health Indicator'].iloc[-1], 'VAR': stock_data['VAR'].iloc[-1]}, stock_data
        else:
            return None, None
    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")
        return None, None

def fetch_future_returns(ticker, start_date, end_date):
    try:
        adjusted_ticker = ticker.replace('.', '-')
        stock_data = yf.Ticker(adjusted_ticker).history(start=start_date, end=end_date)
        if not stock_data.empty:
            stock_data.index = stock_data.index.tz_localize(None)
            start_price = stock_data.iloc[0]['Close']
            end_price = stock_data.iloc[-1]['Close']
            annual_return = (end_price - start_price) / start_price * 100
            return annual_return
        else:
            return None
    except Exception as e:
        print(f"Skipping {ticker} for future returns due to error: {e}")
        return None

def process_year(year, start_date, end_date):
    returns_data = []
    for ticker in sp500_tickers:
        result, stock_data = fetch_and_analyze_data(ticker, start_date, end_date)
        if result and stock_data is not None:
            returns_data.append(result)
            stock_data.to_excel(writer, sheet_name=f'{ticker}_{year}')
    returns_df = pd.DataFrame(returns_data)
    returns_df['P/E Ratio'] = pd.to_numeric(returns_df['P/E Ratio'], errors='coerce')
    returns_df = returns_df.dropna(subset=['P/E Ratio'])
    returns_df = returns_df[returns_df['P/E Ratio'] < 1000]

    average_pe_ratio = returns_df['P/E Ratio'].mean()
    print(f"Average PE Ratio for {year}: ", average_pe_ratio, "\n")

    filtered_df = returns_df[(returns_df['P/E Ratio'] < average_pe_ratio) & (returns_df['Momentum Health Indicator'] > 10)]
    filtered_df_sorted = filtered_df.sort_values(by='Return', ascending=False)
    top_10_stocks = filtered_df_sorted.head(10)
    
    top_10_stocks['2024 Return'] = top_10_stocks['Ticker'].apply(lambda x: fetch_future_returns(x, '2023-12-29', '2024-12-31'))

    top_10_stocks.to_excel(writer, sheet_name=f'Top 10 Stocks {year}', index=False)
    return top_10_stocks

sp500_tickers = get_sp500_tickers()
risk_free_rate = 0.02
writer = pd.ExcelWriter('sp500_data.xlsx', engine='openpyxl')

years = {
    '2023': ('2022-12-31', '2023-12-31')
}

for year, (start_date, end_date) in years.items():
    print(f"Processing data for {year}...")
    top_10_stocks = process_year(year, start_date, end_date)
    print(top_10_stocks[['Ticker', 'Return', 'Std Dev', 'Sharpe Ratio', 'P/E Ratio', 'Momentum Health Indicator', 'VAR', '2024 Return']])

writer.close()


Processing data for 2023...


$GEV: possibly delisted; no price data found  (1d 2022-12-31 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1672462800, endDate = 1703998800")
$SW: possibly delisted; no price data found  (1d 2022-12-31 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1672462800, endDate = 1703998800")
$SOLV: possibly delisted; no price data found  (1d 2022-12-31 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1672462800, endDate = 1703998800")


Average PE Ratio for 2023:  36.0441153149789 



C:\Users\saura\AppData\Local\Temp\ipykernel_5152\2128528740.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_10_stocks['2024 Return'] = top_10_stocks['Ticker'].apply(lambda x: fetch_future_returns(x, '2023-12-29', '2024-12-31'))


    Ticker      Return   Std Dev  Sharpe Ratio  P/E Ratio  \
425   SMCI  238.969731  0.751333      3.153990  15.302736   
404    RCL  165.838665  0.364839      4.490713  23.274824   
76    BLDR  155.455251  0.417931      3.671780  16.053764   
454   UBER  142.783905  0.361867      3.890491  32.837440   
386    PHM  125.444414  0.300122      4.113140   8.518081   
208     GE   92.971734  0.242983      3.743953  35.399610   
261    JBL   90.273934  0.365912      2.412433  14.801798   
287    LII   87.906619  0.291485      2.947210  30.105800   
68    BKNG   74.549864  0.249525      2.907514  32.898220   
155    DHI   69.149812  0.281647      2.384183  10.146443   

     Momentum Health Indicator       VAR  2024 Return  
425                       13.2 -1.466602     7.929359  
404                       14.4 -0.710890    80.581388  
76                        16.4 -0.815027   -14.118849  
454                       12.4 -0.705434    -1.299333  
386                       17.2 -0.584793     6.1